In [117]:
import pandas as pd
import numpy as np
import plotly.express as px
import math as math

# Datos de entrada y calculados-WPI

In [118]:
# Entradas que deben ir en la app
p200 = 80
PI = 25
wPI = 20
# calculados wpi
a_wpi = 0.00364 * (wPI ** 3.35) + 4 * wPI + 11
c_wpi = 0.0514 * (wPI ** 0.465) + 0.5
b_wpi= c_wpi * (-2.313 * (wPI ** 0.14) + 5)
hr_wpi = (0.00364 * (wPI ** 3.35) + 4 * wPI + 11) * 32.44 * math.exp(0.0186 * wPI)
Sopt = (6.752 * (wPI ** 0.147) + 78) / 100

# Datos de entrada y calculados-D60

In [119]:
# Datos de entrada del d60 tmi 
D60 = 0.1
b_D60 = 7.5
# Datos calculados del d60 tmi
a_D60 = 0.8627 * (D60 ** -0.751)
c_D60 = 0.1772 * math.log(D60) + 0.7734
hr_D60 = a_D60 / (D60 + 0.00097)

# Datos de entrada Alfa, beta, gamma, delta (Para calcular las tablas de suelos finos y gruesos)

In [120]:
alpha = 0.3000
beta = 921.4080  
gamma = 150.9908 
delta = 29.8440

# TABLA TMI- MENSUAL

In [121]:
TMI_tabla = pd.DataFrame(
    {
        "mes": [
            "enero",
            "febrero",
            "marzo",
            "abril",
            "mayo",
            "junio",
            "julio",
            "agosto",
            "septiembre",
            "octubre",
            "noviembre",
            "diciembre",
        ],
        "T(°C)": [ # Temperatura promedio mensual
            25.9866666666667,
            26.3933333333333,
            26.2400000000000,
            26.0733333333333,
            26.0800000000000,
            26.0400000000000,
            26.7533333333333,
            27.7266666666667,
            27.4933333333333,
            26.6466666666667,
            25.7200000000000,
            25.5866666666667
        ],
        "Pi(mm)": [ # Precipitación mensual
            182.8800000000000,
            159.9800000000000,
            216.3200000000000,
            192.8000000000000,
            164.5933333333330,
            75.4333333333333,
            38.8933333333333,
            32.4400000000000,
            133.1666666666670,
            229.6666666666670,
            286.9200000000000,
            240.8266666666670
        ],
    }
)

In [122]:
TMI_tabla["i"] = (TMI_tabla["T(°C)"] / 5) ** 1.514

In [123]:
# Este I no  esta aproximado aqui, en excel se aproxima
suma_i = TMI_tabla["i"].sum()
suma_i

149.0119643865716

In [124]:
# Este alfa promedio no me da con los decimales que da en el excel
Alfa_promedio = (0.000000675 * suma_i**3 
                - 0.0000771 * suma_i**2 
                + 0.01792 * suma_i 
                + 0.49239)
Alfa_promedio

3.684115900836029

In [125]:
TMI_tabla["ETPsc(mm/mes)"] = 16 * (10 * TMI_tabla["T(°C)"] / suma_i) ** Alfa_promedio

In [126]:
TMI_tabla["d"] = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [127]:
TMI_tabla["ETPcrr(mm/mes)"] = TMI_tabla["ETPsc(mm/mes)"] * 12 / 12 * TMI_tabla["d"] / 30

In [128]:
TMI_tabla["TMI (1)"] = 100 * (TMI_tabla["Pi(mm)"] / TMI_tabla["ETPcrr(mm/mes)"] - 1)

In [129]:
TMI_tabla["TMI (2)"] = 75 * (TMI_tabla["Pi(mm)"] / TMI_tabla["ETPcrr(mm/mes)"] - 1) + 10

In [130]:
TMI_tabla

,mes,T(°C),Pi(mm),i,ETPsc(mm/mes),d,ETPcrr(mm/mes),TMI (1),TMI (2)
0,enero,25.986667,182.880000,12.125280,124.148769,31,128.287061,42.555296,41.916472
1,febrero,26.393333,159.980000,12.413713,131.457954,28,122.694090,30.389328,32.791996
2,marzo,26.240000,216.320000,12.304689,128.666218,31,132.955092,62.701553,57.026165
3,abril,26.073333,192.800000,12.186556,125.680986,30,125.680986,53.404271,50.053203
4,mayo,26.080000,164.593333,12.191274,125.799417,31,129.992731,26.617337,29.963002
5,junio,26.040000,75.433333,12.162976,125.090051,30,125.090051,-39.696776,-19.772582
6,julio,26.753333,38.893333,12.670961,138.185650,31,142.791838,-72.762215,-44.571662
7,agosto,27.726667,32.440000,13.375391,157.630242,31,162.884584,-80.084058,-50.063043
8,septiembre,27.493333,133.166667,13.205343,152.798069,30,152.798069,-12.847939,0.364046
9,octubre,26.646667,229.666667,12.594553,136.166716,31,140.705606,63.224958,57.418718


 # Calculo de tabla TMI- PARA SUELOS FINOS

In [131]:
TMI_tabla_finos = pd.DataFrame(
    {
        "mes": [
            "enero",
            "febrero",
            "marzo",
            "abril",
            "mayo",
            "junio",
            "julio",
            "agosto",
            "septiembre",
            "octubre",
            "noviembre",
            "diciembre",
        ],           
    }
)

In [132]:
TMI_tabla_finos["h(1)"] = alpha * (np.exp(beta / (TMI_tabla["TMI (1)"] + gamma)) + delta)

In [133]:
TMI_tabla_finos["h(2)"] = alpha * (np.exp(beta / (TMI_tabla["TMI (2)"] + gamma)) + delta)

In [134]:
TMI_tabla_finos["Ch(h1)"] = 1 - np.log(1 + TMI_tabla_finos["h(1)"] / hr_wpi) / np.log(1 + 1000000 / hr_wpi)

In [135]:
TMI_tabla_finos["Ch(h2)"] = 1 - np.log(1 + TMI_tabla_finos["h(2)"] / hr_wpi) / np.log(1 + 1000000 / hr_wpi)

In [136]:
TMI_tabla_finos["Sr(1)"] = TMI_tabla_finos["Ch(h1)"] / ((np.log(np.exp(1) + (TMI_tabla_finos["h(1)"] / a_wpi) ** b_wpi) ** c_wpi))

In [137]:
TMI_tabla_finos["Sr(2)"] = TMI_tabla_finos["Ch(h2)"] / ((np.log(np.exp(1) + (TMI_tabla_finos["h(2)"] / a_wpi) ** b_wpi)) ** c_wpi)


In [138]:
TMI_tabla_finos["Mr 1/Mropt(Finos)"] = 10 ** (
    -0.3123 +
    (0.3 - -0.3123) /
    (1 + np.exp(
        np.log(-0.3 / -0.3123) + 6.8157 * (TMI_tabla_finos["Sr(1)"] - Sopt)
    ))
)

In [139]:
TMI_tabla_finos["Mr 2/Mropt(Finos)"] = 10 ** (
    -0.3123 +
    (0.3 - (-0.3123)) /
    (1 + np.exp(
        np.log(-0.3 / -0.3123) + 6.8157 * (TMI_tabla_finos["Sr(2)"] - Sopt)
    ))
)

In [140]:
TMI_tabla_finos

,mes,h(1),h(2),Ch(h1),Ch(h2),Sr(1),Sr(2),Mr 1/Mropt(Finos),Mr 2/Mropt(Finos)
0,enero,44.000254,44.557157,0.998887,0.998873,0.943828,0.943161,0.869146,0.870495
1,febrero,57.184605,54.085477,0.998555,0.998633,0.928422,0.931973,0.901125,0.893607
2,marzo,31.326556,34.119658,0.999207,0.999136,0.959381,0.955891,0.838585,0.845289
3,abril,36.174584,38.298819,0.999085,0.999031,0.953346,0.950735,0.850234,0.855357
4,mayo,62.679231,57.765326,0.998416,0.998540,0.922232,0.927761,0.914438,0.902533
5,junio,1191.079592,345.244918,0.971795,0.991423,0.535372,0.723401,1.778711,1.418119
6,julio,39130.662445,1736.261245,0.635592,0.960060,0.186099,0.484876,1.972496,1.835585
7,agosto,132024.232621,2775.648161,0.409900,0.939373,0.104094,0.427991,1.982167,1.883392
8,septiembre,245.463519,141.079849,0.993866,0.996452,0.774305,0.846896,1.286354,1.094686
9,octubre,31.092082,33.910562,0.999213,0.999142,0.959676,0.956152,0.838023,0.844786


 # Calculo de tabla TMI- PARA SUELOS GRUESOS

In [141]:
TMI_tabla_gruesos = pd.DataFrame(
    {
        "mes": [
            "enero",
            "febrero",
            "marzo",
            "abril",
            "mayo",
            "junio",
            "julio",
            "agosto",
            "septiembre",
            "octubre",
            "noviembre",
            "diciembre",
        ],           
    }
)

In [142]:
TMI_tabla_gruesos["h(1)"]= alpha * (np.exp(beta / (TMI_tabla["TMI (1)"] + gamma)) + delta)

In [143]:
TMI_tabla_gruesos["h(2)"] = alpha * (np.exp(beta / (TMI_tabla["TMI (2)"] + gamma)) + delta)

In [144]:
TMI_tabla_gruesos["Ch(h1)"] = 1 - np.log(1 + TMI_tabla_gruesos["h(1)"] / hr_D60) / np.log(1 + 1000000 / hr_D60)

In [145]:
TMI_tabla_gruesos["Ch(h2)"] = 1 - np.log(1 + TMI_tabla_gruesos["h(2)"] / hr_D60) / np.log(1 + 1000000 / hr_D60)

In [146]:
TMI_tabla_gruesos["Sr(1)"] = TMI_tabla_gruesos["Ch(h1)"] / ((np.log(np.exp(1) + (TMI_tabla_gruesos["Ch(h1)"] / a_D60) ** b_D60)) ** c_D60)

In [147]:
TMI_tabla_gruesos["Sr(2)"] = TMI_tabla_gruesos["Ch(h2)"] / ((np.log(np.exp(1) + (TMI_tabla_gruesos["Ch(h2)"] / a_D60)**b_D60))**c_D60)

In [148]:
TMI_tabla_gruesos["Mr 1/Mropt(Gruesos)"] = 10 ** (
    -0.3123 + 
    (0.3 - -0.3123) / 
    (1 + np.exp(
        np.log(-0.3 / -0.3123) + 
        6.8157 * (TMI_tabla_gruesos["Sr(1)"] - Sopt)
    ))
)

In [149]:
TMI_tabla_gruesos["Mr 2/Mropt(Gruesos)"] = 10 ** (
    -0.3123 + 
    (0.3 - -0.3123) / 
    (1 + np.exp(
        np.log(-0.3 / -0.3123) + 
        6.8157 * (TMI_tabla_gruesos["Sr(2)"] - Sopt)
    ))
)

In [151]:
TMI_tabla_gruesos

,mes,h(1),h(2),Ch(h1),Ch(h2),Sr(1),Sr(2),Mr 1/Mropt(Gruesos),Mr 2/Mropt(Gruesos)
0,enero,44.000254,44.557157,0.934713,0.934107,0.934713,0.934107,0.887865,0.889131
1,febrero,57.184605,54.085477,0.921263,0.924267,0.921262,0.924266,0.916546,0.910034
2,marzo,31.326556,34.119658,0.949595,0.946121,0.949595,0.946121,0.857610,0.864531
3,abril,36.174584,38.298819,0.943640,0.941137,0.943639,0.941137,0.869528,0.874611
4,mayo,62.679231,57.765326,0.916148,0.920710,0.916148,0.920709,0.927774,0.917752
5,junio,1191.079592,345.244918,0.673298,0.788720,0.673298,0.788720,1.536987,1.248041
6,julio,39130.662445,1736.261245,0.325887,0.636622,0.325887,0.636622,1.937506,1.614242
7,agosto,132024.232621,2775.648161,0.203645,0.590451,0.203645,0.590451,1.969649,1.698142
8,septiembre,245.463519,141.079849,0.818148,0.862337,0.818148,0.862337,1.169799,1.055421
9,octubre,31.092082,33.910562,0.949893,0.946377,0.949892,0.946377,0.857022,0.864018
